In [23]:
from IPython.core.display import HTML
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from pylab import *
import pysis
import os
import scipy.optimize as sco
import time
import ast
from tkinter import *
from PIL import Image, ImageTk
%matplotlib qt



In [9]:
def gaussian(x,mean,sigma,A):
    
    return A*np.exp( -((x-mean)**2.) / (2.*sigma**2.) )

def powerlaw(x,a, k, c):
    
    return a*x**k + c

vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 

In [18]:
def write_cubes(master_directory,file_name):
    viewing_geometries = os.listdir(master_directory)
    file = open(file_name,'w')
    for folder in viewing_geometries:
        print(f'Working on folder {folder}')
        for cube in os.listdir(f'{master_directory}/{folder}'):
            if cube[0:2] != 'CM' or 'nav' in cube or 'BKGD' in cube:
                continue
            file.write(f'{master_directory}/{folder}/{cube}\n')
    return file_name

In [19]:
def analyze_cube(cube,nav,l,w):
    """
    Given cube data (cube) and navigation information (nav) with dimensions x and y, returns:
    
    1) max/min 2 micron amplitude
    2) max/min sigma
    3) max/min mean 
    4) max/min incidence
    5) max/min emission
    6) max/min phase
    7) max/min lat
    8) max/min lon
    """
    
    count=0
    band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 
    
    info = np.full((40,l,w),None)
    #0: key (0 = good, 1 = off disk, 2 = low resolution, 3 = bad powerlaw, 4 = dark, 5 = bad incidence/emission), 6 = completely failed 
    #1-6: best fit (1-3: gaussian mean,sigma,amp; 4-6: plaw amp,exp,const)
    #7-32: raw data channel 60:85
    #32-40: phase,emission,incidence, lat, lon, resolution,spacecraft azimuth, solar azimuth
    

    info[7:32]=cube[60:85]
    info[32:35] = nav[1:4]
    info[35:38] = nav[6:9]
    info[38:] = nav[16:18]

    for i in range(l):
        for j in range(w):
            
            if nav[6,i,j] < -90:  
                count+=1
                info[0,i,j] = 1
                continue
            elif nav[8,i,j]>100000:
                info[0,i,j] = 2
            elif cube[69,i,j]<=0.02:
                info[0,i,j] = 4
            elif nav[2,i,j]>70 or nav[3,i,j]>70:
                info[0,i,j] = 5
            
            
            

            try:
                #Fit power law
                p_fit, p_cov = sco.curve_fit(powerlaw, vims_wave[band_channels], cube[band_channels, i,j], p0 = (.005,-2,0), maxfev = 10000)
                p = powerlaw(vims_wave, *p_fit)

                #Subtract power law
                new_data = cube[:,i,j] - p

                #Fit gaussian
                fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],new_data[64:85], \
                                    p0=(2.08,.125,.1), maxfev=100000)

                info[1,i,j] = fit[0]
                info[2,i,j] = fit[1]
                info[3,i,j] = fit[2]
                info[4,i,j] = p_fit[0]
                info[5,i,j] = p_fit[1]
                info[6,i,j] = p_fit[2]
                info[0,i,j] = 0


            except:
                #If powerlaw fit fails, directly fit gaussian
                info[0,i,j] = 3
                try:
                    fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],cube[64:85,i,j], p0=(2.08,.125,.1), maxfev=1000000)
                    info[1,i,j] = fit[0]
                    info[2,i,j] = fit[1]
                    info[3,i,j] = fit[2]
                except:
                    info[0,i,j]=6
                    break


    return info

In [20]:
def create_cube_parser(cube_list,destination='cube_info'):
    '''
    Parameters:
    
    master_directory (str): Path to folder with all viewing geometries.
    
    Creates text file with information about each cube (assuming max resolution < 100km): 
    
    1) folder 
    2) name 
    3) max/min 2 micron amplitude
    4) max/min sigma
    5) max/min mean
    6) dimensions
    7) max/min incidence
    8) max/min emission
    9) max/min phase
    10) max/min lat
    11) max/min lon
    
    '''
    count = 0

    skipped=set()
    file = open(cube_list,'r')
    
    while True:
        
        
        cube_path = file.readline().strip()
        
        if not cube_path:
            break
            
        nav = pysis.CubeFile(f'{cube_path[:-4]}_nav.cub').data


        print(f'Working on: {cube_path}')
            
            
        folder = cube_path.split('/')[-2]
        name = cube_path.split('/')[-1]
        
        data = pysis.CubeFile(cube_path).data

        lbl = pysis.CubeFile(cube_path).label
        length=lbl['IsisCube']['Instrument']['SwathLength']
        width=lbl['IsisCube']['Instrument']['SwathWidth']
        t0=time.time()
        info = analyze_cube(data,nav,length,width)
        t1 = time.time()
        length = f'0{length}' if length < 10 else length
        width = f'0{width}' if width < 10 else width
        np.save(os.path.join(destination, f'{name}-{length}x{width}.npy'),info)
          
    file.close()  

    return skipped

In [21]:
def create_database(master_directory,destination='cube_info',file_name='cube_list.txt'):
    create_cube_parser(write_cubes(master_directory,file_name),destination)

In [22]:
#on test folder (one viewing geometry, 11 cubes): 35.3 s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

create_database('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test')

Working on folder 2009189T194600_2009189T235739
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625786407_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625776676_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777485_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777418_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777687_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625780003_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777215_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777620_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_162577842

/home/alanyu/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625788353_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625776406_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625785908_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625783874_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625778092_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625778227_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625776946_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625777553_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2009189T194600_2009189T235739/CM_1625785412_1_ir.cub
Working on: /home/alanyu/Dropbox (MIT

In [ ]:
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir_nav.cub').data
data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').label
analyze_cube(data,nav,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])

In [ ]:
write_cubes('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

In [ ]:
create_cube_parser('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

In [ ]:
def read_database(file_name):
    data = {}
    
    file = open(file_name,'r')
    while True:
        try:
            temp = ast.literal_eval(file.readline())
            name = temp['name']
            del(temp['name'])
            data.update({name:temp})
        except Exception as e:
            #print(e)
            break
            
    file.close()
    
    return data


In [ ]:
def search(database):
    parameters = input('Enter parameters as tuples separated by whitespace. Use None to specify no filter for that parameter. \nOrdering: Real Amplitude, Mean, Sigma, Gaussian Amplitude, Incidence, Emission, Phase, Latitude, Longitude.')
    parameters = [eval(x) for x in parameters]#['None', 'None', 'None','None','None','None','None','None', '(180,360)']]
    
    assert len(parameters) == 10, 'Incorrect number of parameters specified'
    code = {0:'real_amplitude',1:'mean',2:'sigma',3:'amplitude',4:'incidence',5:'emission',6:'phase',7:'lat',8:'lon'}
    out = {}
    for cube in database:
        for i in range(len(parameters)):
            if parameters[i] is not None and (database[cube][f'min_{code[i]}']<parameters[i][0] or \
                                              database[cube][f'max_{code[i]}']>parameters[i][1]):
                break
        else:
             out.update({cube:database[cube]})
                
    return out

In [ ]:

%timeit create_cube_parser('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test')
#With fitting
#one directory: 2005359T100445_2005360T101919
#19.2 s ± 1.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

#No fitting, but with loops
#16.9 s ± 566 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#control: just looping through and loading files, no analysis
#15.8 s ± 380 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
d

In [ ]:
class Window(Frame):
    def __init__(self, master = None):
        Frame.__init__(self, master)
        self.master = master
        self.init_window()
    def printtext(e):
        def p():
            print(e.get())
        return p
    def init_window(self):
        self.master.title('GUI')
        
        self.pack(fill=BOTH,expand=1)
        
        #quitButton = Button(self,text='QUIT',command=self.client_exit)
        
        #quitButton.place(x=0,y=0)
        
        menu = Menu(self.master)
        self.master.config(menu=menu)
        
        file = Menu(menu)
        file.add_command(label='Save')
        file.add_command(label='Exit',command=self.client_exit)
        menu.add_cascade(label='File',menu=file)
        
        edit=Menu(menu)
        edit.add_command(label='Show Image',command = self.showImg)
        edit.add_command(label='Show Text',command=self.showText)
        menu.add_cascade(label='Edit',menu=edit)
        
        e=Entry(self)
        e.pack()
        e.focus_set()
        b=Button(self,text='ok')
        b.pack(side='bottom')
        
    
    def showImg(self):
        load=Image.open('2005_3d.png')
        render = ImageTk.PhotoImage(load)
        
        img=Label(self,image=render)
        img.image=render
        img.place(x=0,y=0)
        
    def showText(self):
        text=Label(self,text='Hello World')
        text.pack()
        
    def client_exit(self):
        self.master.destroy()
        
root = Tk()
root.geometry('400x300')
app = Window(root)

root.mainloop()

In [ ]:
import tkinter
class App():
    def __init__(self):
        self.root = tkinter.Tk()
        button = tkinter.Button(self.root, text = 'root quit', command=self.quit)
        button.pack()
        self.root.mainloop()

    def quit(self):
        self.root.destroy()

app = App()

In [40]:
%timeit test = np.load('/home/alanyu/Dropbox (MIT)/VIMS_UROP/cube_info/2005359T100445_2005360T101919-CM_1514280052_1_ir.cub-64x64.npy',allow_pickle=True)
test[20]

7.81 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


array([[-0.0009496157290413976, 0.00018700241344049573,
        0.00018519342120271176, ..., -0.0006614071317017078,
        -0.0006621116772294044, -0.000663038925267756],
       [0.00018211669521406293, 0.00018036890833172947,
        0.00017863961693365127, ..., 0.00012762020924128592,
        0.000915833399631083, -0.0006611858843825758],
       [0.00017566338647156954, 0.00017399003263562918,
        0.0001723362656775862, ..., 0.00012380917905829847,
        0.00012391257041599602, 0.00012405408779159188],
       ...,
       [-6.334969657473266e-05, -0.0010361031163483858,
        -6.2333740061149e-05, ..., 0.029725924134254456,
        0.023611851036548615, 0.01445066835731268],
       [-6.727128493366763e-05, -6.67211934342049e-05,
        -6.618737825192511e-05, ..., 0.025243433192372322,
        0.019100036472082138, 0.011433631181716919],
       [-7.122118404367939e-05, -7.063517841743305e-05,
        -7.006782107055187e-05, ..., 0.02303570695221424,
        0.01532457023859

480 ms ± 28.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
